# GPU accelerated solution using NVIDIA RAPIDS cudf and cuml
# Data loading, preprocessing and feature engineering takes less than 3min in GPU.

In [ ]:
import pandas
import numpy as np
import cudf as pd
import cupy as cp

import glob
import os
import gc
import time

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
from scipy.optimize import minimize


import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
from catboost import Pool, CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import cuml
from cuml.neighbors import KNeighborsRegressor
from cuml import LinearRegression
from cuml import Ridge
from cuml.ensemble import RandomForestRegressor


path_submissions = '/'

target_name = 'target'
scores_folds = {}

def convert_to_32bit(df):
    for f in df.columns:
        if df[f].dtype == 'int64':
            df[f] = df[f].astype('int32')
        if df[f].dtype == 'float64':
            df[f] = df[f].astype('float32')
    return df

In [ ]:
#set clip values
low_clip = 0
upper_clip = 1e10

# Loading train and test sets

In [ ]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')

train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)

train['is_train'] = 1
test['is_train'] = 0

train = convert_to_32bit(train)
test = convert_to_32bit(test)

print( train.shape )
print( test.shape )

In [ ]:
print(train.head(20))

In [ ]:
print(test.head(20))

# Checking how many stock_id there are in train and test

In [ ]:
train_stock_ids = train['stock_id'].to_pandas().unique()
test_stock_ids = test['stock_id'].to_pandas().unique()
print( 'Sizes:', len(train_stock_ids), len(test_stock_ids) )
print( 'Train stocks:', train_stock_ids )
print( 'Test stocks:', test_stock_ids )

In [ ]:
def transform(df, groupby='time_id', feat='price', agg='mean' ):
    return df.merge( 
        df.groupby(groupby)[feat].agg(agg).reset_index().rename({feat:feat+'_'+agg}, axis=1),
        on=groupby,
        how='left' 
    )

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df = convert_to_32bit(df)
    df = df.sort_values(['time_id','seconds_in_bucket']).reset_index(drop=True)
    
    # Calculate Wap
    df['wap1'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    df['wap2'] = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    df['wap3'] = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    df['wap4'] = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    
    
    # Calculate log returns
    df['log_return1'] = df['wap1'].log()
    df['log_return1'] = df['log_return1'] - df.groupby(['time_id'])['log_return1'].shift(1).reset_index(drop=True)

    df['log_return2'] = df['wap2'].log()
    df['log_return2'] = df['log_return2'] - df.groupby(['time_id'])['log_return2'].shift(1).reset_index(drop=True)

    df['log_return3'] = df['wap3'].log()
    df['log_return3'] = df['log_return3'] - df.groupby(['time_id'])['log_return3'].shift(1).reset_index(drop=True)

    df['log_return4'] = df['wap4'].log()
    df['log_return4'] = df['log_return4'] - df.groupby(['time_id'])['log_return4'].shift(1).reset_index(drop=True)
    
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['log_return1_sqr'] = df['log_return1'] ** 2
    df['log_return2_sqr'] = df['log_return2'] ** 2
    df['log_return3_sqr'] = df['log_return3'] ** 2
    df['log_return4_sqr'] = df['log_return4'] ** 2
    
    df = transform(df, groupby='time_id', feat='wap1', agg='median' )
    df = transform(df, groupby='time_id', feat='wap2', agg='median' )
    df = transform(df, groupby='time_id', feat='wap3', agg='median' )
    df = transform(df, groupby='time_id', feat='wap4', agg='median' )
    df = transform(df, groupby='time_id', feat='log_return1', agg='median' )
    df = transform(df, groupby='time_id', feat='log_return2', agg='median' )
    df = transform(df, groupby='time_id', feat='log_return3', agg='median' )
    df = transform(df, groupby='time_id', feat='log_return4', agg='median' )
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': ['sum', 'std', 'min','max'],
        'wap2': ['sum', 'std', 'min','max'],
        'wap3': ['sum', 'std', 'min','max'],
        'wap4': ['sum', 'std', 'min','max'],
        'log_return1_sqr': ['sum', 'std', 'min','max'],
        'log_return2_sqr': ['sum', 'std', 'min','max'],
        'log_return3_sqr': ['sum', 'std', 'min','max'],
        'log_return4_sqr': ['sum', 'std', 'min','max'],
        'wap_balance': ['sum', 'mean', 'min','max'],
        'price_spread':['sum', 'mean', 'min','max'],
        'price_spread2':['sum', 'mean', 'min','max'],
        'bid_spread':['sum', 'mean', 'min','max'],
        'ask_spread':['sum', 'mean', 'min','max'],
        'total_volume':['sum', 'mean', 'min','max'],
        'volume_imbalance':['sum', 'mean', 'min','max'],
        "bid_ask_spread":['sum',  'mean', 'min','max'],
        
        "wap1_median":['sum',  'mean', 'min','max'],
        "wap2_median":['sum',  'mean', 'min','max'],
        "wap3_median":['sum',  'mean', 'min','max'],
        "wap4_median":['sum',  'mean', 'min','max'],
        "log_return1_median":['mean',  'std', 'min','max'],
        "log_return2_median":['mean',  'std', 'min','max'],
        "log_return3_median":['mean',  'std', 'min','max'],
        "log_return4_median":['mean',  'std', 'min','max'],
    }
    create_feature_dict_time = {
        'log_return1_sqr': ['sum', 'std', 'min','max'],
        'log_return2_sqr': ['sum', 'std', 'min','max'],
        'log_return3_sqr': ['sum', 'std', 'min','max'],
        'log_return4_sqr': ['sum', 'std', 'min','max'],
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature.columns = [col + '_' + str(seconds_in_bucket) for col in df_feature.columns]
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    # Drop tmp columns
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['stock_id'] = str(stock_id) + '-'

    df_feature['row_id'] = df_feature['stock_id'] + df_feature['time_id_'].astype(str)
    
    return df_feature

In [ ]:
%%time

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df = convert_to_32bit(df)
    df = df.sort_values('time_id').reset_index(drop=True)

    df['log_return'] = df['price'].log()
    df['log_return'] = df['log_return'] - df.groupby(['time_id'])['log_return'].shift(1).reset_index(drop=True)
    df['log_return_sqr'] = df['log_return'] ** 2
    
    df['amount']=df['price']*df['size']
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return_sqr': ['sum', 'std','max', 'min'],
        'seconds_in_bucket':['nunique','std', 'mean','max', 'min'],
        'size':['sum', 'nunique','std','max', 'min'],
        'order_count':['sum','nunique','max','min','std'],
        'amount':['sum','std','max','min'],
    }
    create_feature_dict_time = {
        'log_return_sqr': ['sum', 'std','max','min'],
        'seconds_in_bucket':['nunique'],
        'size':['sum','mean','std','min','max'],
        'order_count':['sum','mean','std','min','max'],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature.columns = [col + '_' + str(seconds_in_bucket) for col in df_feature.columns]
        return df_feature

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    df = df.sort_values(['time_id','seconds_in_bucket']).reset_index(drop=True)
    
    df = transform(df, groupby='time_id', feat='price', agg='mean' )
    df = transform(df, groupby='time_id', feat='price', agg='sum' )
    df = transform(df, groupby='time_id', feat='size', agg='mean' )
    df['price_dif'] = ((df['price'] - df.groupby(['time_id'])['price'].shift(1).reset_index(drop=True)) / df['price']).fillna(0.)
    df['tendencyV'] = df['size'] * df['price_dif']
    df['f_max'] = 1 * (df['price'] >= df['price_mean'])
    df['f_min'] = 1 * (df['price'] < df['price_mean'])
    df['df_max'] = 1 * (df['price_dif'] >= 0)
    df['df_min'] = 1 * (df['price_dif'] < 0)
    df['abs_dif'] = (df['price'] - df['price_mean']).abs()
    df['price_sqr'] = df['price']**2
    df['size_dif'] = (df['size'] - df['size_mean']).abs()
    df['size_sqr'] = df['size']**2
    df['iqr_p25'] = df.groupby(['time_id'])['price'].quantile(0.15).reset_index(drop=True)
    df['iqr_p75'] = df.groupby(['time_id'])['price'].quantile(0.85).reset_index(drop=True)
    df['iqr_p_v25'] = df.groupby(['time_id'])['size'].quantile(0.15).reset_index(drop=True)
    df['iqr_p_v75'] = df.groupby(['time_id'])['size'].quantile(0.85).reset_index(drop=True)

    df = transform(df, groupby='time_id', feat='price_dif', agg='std' )
    df = transform(df, groupby='time_id', feat='tendencyV', agg='std' )
    df = transform(df, groupby='time_id', feat='f_max', agg='std' )
    df = transform(df, groupby='time_id', feat='f_min', agg='std' )
    df = transform(df, groupby='time_id', feat='df_max', agg='std' )
    df = transform(df, groupby='time_id', feat='df_min', agg='std' )
    df = transform(df, groupby='time_id', feat='size_dif', agg='std' )
    
    dt = df.groupby('time_id')[['tendencyV','price','price_dif','f_max','f_min','df_max','df_min','abs_dif','price_sqr','size_dif','size_sqr','iqr_p25','iqr_p75','iqr_p_v25','iqr_p_v75'
                                 ,'price_dif_std','tendencyV_std','f_max_std','f_min_std','df_max_std','df_min_std','size_dif_std'
                               ]].agg(
        {
            'tendencyV':['sum','std','max', 'min'],
            'price':['mean','std','max', 'min'],
            'price_dif':['mean','std','max', 'min'],
            'f_max':['mean','std','max', 'min'],
            'f_min':['mean','std','max', 'min'],
            'df_max':['mean','std','max', 'min'],
            'df_min':['mean','std','max', 'min'],
            'abs_dif':['median','std','max', 'min'],
            'price_sqr':['sum','std','max', 'min'],
            'size_dif':['median','std','max', 'min'],
            'size_sqr':['sum','std','max', 'min'],
            'iqr_p25':['mean','std','max', 'min'],
            'iqr_p75':['mean','std','max', 'min'],
            'iqr_p_v25':['mean','std','max', 'min'],
            'iqr_p_v75':['mean','std','max', 'min'],
            'price_dif_std':['mean','std','max', 'min'],
            'tendencyV_std':['mean','std','max', 'min'],
            'f_max_std':['mean','std','max', 'min'],
            'f_min_std':['mean','std','max', 'min'],
            'df_max_std':['mean','std','max', 'min'],
            'df_min_std':['mean','std','max', 'min'],
            'size_dif_std':['mean','std','max', 'min']
        }
    )

    
    dt.columns = [i+'_'+j for i, j in dt.columns] 
    df_feature = df_feature.merge(dt, left_on='time_id_', right_index=True, how='left')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    # Drop tmp columns
    df_feature = df_feature.sort_values(['time_id_' ]).reset_index(drop=True)
    
    stock_id = file_path.split('=')[1]
    df_feature['stock_id'] = str(stock_id) + '-'
    df_feature['row_id'] = df_feature['stock_id'] + df_feature['time_id_'].astype(str)
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200', 'time_id_','time_id__100','stock_id'], axis = 1, inplace = True)

    fnames = ['trade_' + f for f in df_feature.columns]
    fnames[-1] = 'row_id'
    df_feature.columns = fnames

    return df_feature

# Process all train .parquet files. Create features using cudf (GPU)
# Note cudf speed to load and apply all feature engineering in all train set stocks.

In [ ]:
%%time
DF_TRAIN = []
for stock_id in tqdm(train_stock_ids, disable=True):
    df_tmp = pd.merge( 
        book_preprocessor(data_dir + "book_train.parquet/stock_id=" + str(stock_id)),
        trade_preprocessor(data_dir + "trade_train.parquet/stock_id=" + str(stock_id)),
        on = 'row_id',
        how = 'left'
    )
    df_tmp['stock_id'] = stock_id
    df_tmp = convert_to_32bit(df_tmp) # to save memory
    #df_tmp.to_parquet( 'train_parquet/'+str(stock_id)+'.parquet' )
    DF_TRAIN.append(df_tmp)

# Concatenate all stock_id in the same dataframe
DF_TRAIN = pd.concat(DF_TRAIN, ignore_index=True )
_ = gc.collect()

# Flag to filter train/test rows
DF_TRAIN['is_test'] = 0
DF_TRAIN.shape

# Process all test .parquet files. Create features using cudf (GPU)

In [ ]:
%%time
DF_TEST = []
for stock_id in tqdm(test_stock_ids, disable=True):
    df_tmp = pd.merge( 
        book_preprocessor(data_dir + "book_test.parquet/stock_id=" + str(stock_id)),
        trade_preprocessor(data_dir + "trade_test.parquet/stock_id=" + str(stock_id)),
        on = 'row_id',
        how = 'left'
    )
    df_tmp['stock_id'] = stock_id
    df_tmp = convert_to_32bit(df_tmp) # to save memory
    #df_tmp.to_parquet( 'test_parquet/'+str(stock_id)+'.parquet' )
    DF_TEST.append(df_tmp)
    
# Concatenate all stock_id in the same dataframe
DF_TEST = pd.concat(DF_TEST, ignore_index=True )
_ = gc.collect()

# Flag to filter train/test rows
DF_TEST['is_test'] = 1
DF_TEST.shape

In [ ]:
TRAIN = pd.concat( [DF_TRAIN, DF_TEST] ).sort_values(['stock_id','time_id_']).reset_index(drop=True)

del DF_TRAIN, DF_TEST
_ = gc.collect()
TRAIN.shape

In [ ]:
%%time

def get_time_stock(df_):
    vol_cols = ['log_return1_sqr_sum_500', 'log_return2_sqr_sum_500', 'log_return3_sqr_sum_500', 'log_return4_sqr_sum_500', 'trade_log_return_sqr_sum', 'trade_log_return_sqr_std', 'trade_seconds_in_bucket_nunique' ]

    df = df_.copy()
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_stock_id.columns = ['_'.join(col) + '_stock' for col in df_stock_id.columns]

    df_time_id = df.groupby(['time_id_'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_time_id.columns = ['_'.join(col)+ '_time' for col in df_time_id.columns]
    
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id_'], right_on = ['time_id___time'])
    df.drop(['stock_id__stock', 'time_id___time'], axis = 1, inplace = True)
    return df

TRAIN_ = get_time_stock(TRAIN)
TRAIN_.drop(['stock_id','time_id_'], axis = 1, inplace = True)
print(TRAIN_.shape)
print(TRAIN_.head())

In [ ]:
train = train.merge(TRAIN_, on='row_id', how='left' )
test  = test.merge(TRAIN_, on='row_id', how='left' )

del TRAIN_, TRAIN
_ = gc.collect()

train.shape, test.shape

In [ ]:
train.head()

# Now time to calculate correlation between all stock. The best way is using a correlation matrix, so first pivot all target variables by stock_id, then just calculate the correlation matrix.
# To Find correlated stocks use Kmeans algorithm on the correlation matrix. This procedure is a bit leak because it not being processed using crossvalidation, but it won't leak much since only 6 clusters are being calculated.

In [ ]:
%%time
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns=['stock_id'], values=['target']).fillna(0.)
corr = train_p.corr()

km = cuml.KMeans(n_clusters=6, max_iter=2000, n_init=5).fit(corr)
df = pd.DataFrame( {'stock_id': [ f[1] for f in corr.columns ], 'cluster': km.labels_} )
df = convert_to_32bit(df)

train = train.merge(df, on='stock_id', how='left')
test = test.merge(df, on='stock_id', how='left')


del train_p, df, corr, km
_ = gc.collect()

# Clusters found
train.groupby('cluster')['time_id'].agg('count')

In [ ]:
# replace by order sum (tau)
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_nunique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_nunique'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_nunique_mean_time'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_nunique_mean_time'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_nunique_std_time'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_nunique_std_time'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_nunique_max_time'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_nunique_max_time'] )
train['size_tau_100'] = np.sqrt( 1/ train['trade_seconds_in_bucket_nunique_min_time'] )
test['size_tau_100'] = np.sqrt( 1/ test['trade_seconds_in_bucket_nunique_min_time'] )

train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )

train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )

train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# delta tau
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

In [ ]:
matTrain = []
matTest = []

# 6 clusters
for ind in range(train.cluster.max()+1):
    print(ind)
    newDf = train.loc[train['cluster']==ind].copy()
    newDf = newDf.groupby(['time_id']).agg('mean')
    newDf.loc[:,'stock_id'] = 127+ind
    matTrain.append ( newDf )
    
    newDf = test.loc[test['cluster']==ind].copy()
    newDf = newDf.groupby(['time_id']).agg('mean')
    newDf.loc[:,'stock_id'] = 127+ind
    matTest.append ( newDf )
    
matTrain = pd.concat(matTrain).reset_index()
matTrain.drop(columns=['target'],inplace=True)

matTest = pd.concat(matTest).reset_index()

matTrain.shape, matTest.shape

In [ ]:
matTest = pd.concat([matTest , matTrain.loc[matTrain.time_id==5]])
matTrain = matTrain.pivot(index='time_id', columns='stock_id')
matTrain.columns = [x[0]+'_stock'+str(int(x[1])) for x in matTrain.columns]
matTrain.reset_index(inplace=True)

matTest = matTest.pivot(index='time_id', columns='stock_id')
matTest.columns = [x[0]+'_stock'+str(int(x[1])) for x in matTest.columns]
matTest.reset_index(inplace=True)

matTrain.shape, matTest.shape

In [ ]:
cluster_cols = ['size_tau2', 'wap1_sum', 'wap2_sum', 'wap3_sum', 'wap4_sum', 'log_return1_sqr_sum', 'log_return2_sqr_sum', 
    'log_return3_sqr_sum', 'log_return4_sqr_sum', 'total_volume_sum', 'trade_size_sum', 'trade_order_count_sum',
    'price_spread_sum', 'bid_spread_sum', 'ask_spread_sum', 'volume_imbalance_sum', 'bid_ask_spread_sum']

cluster_labels = ['_stock127', '_stock128', '_stock129', '_stock130', '_stock131', '_stock132']

kfeatures = [
    'time_id'
]

for cols in cluster_cols:
    for label in cluster_labels:
        kfeatures.append(cols+label)
    
print(kfeatures)
        
matTrain = convert_to_32bit(matTrain)
matTest = convert_to_32bit(matTest)

train = pd.merge(train,matTrain[kfeatures],how='left',on='time_id')
test = pd.merge(test,matTest[kfeatures],how='left',on='time_id')
_ = gc.collect()

print(train.shape, test.shape)

In [ ]:
# train=train[~(train["stock_id"]==31)].reset_index(drop=True)
# _= gc.collect()

train = convert_to_32bit(train)
test  = convert_to_32bit(test)
_= gc.collect()

train.shape, test.shape

In [ ]:
y_target = train.target.to_pandas() #need to be numpy or pandas for sklearn 
time_id = train.time_id.to_pandas()
NFOLD = 5

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


train_pandas = train.to_pandas()
test_pandas = test.to_pandas()

# Target min and max values
np.min(y_target), np.max(y_target)

# Neuron Network

## Data preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

In [ ]:
def apply_quantile(train, test):
    colNames = [col for col in list(train.columns)
                if col not in {"stock_id", "time_id", "target", "target_sqr", "row_id", 'is_train'}]

    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    train_nn=train[colNames].copy()
    test_nn=test[colNames].copy()
    for col in colNames:
        #print(col)
        qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
        train_nn[col] = qt.fit_transform(train_nn[[col]])
        test_nn[col] = qt.transform(test_nn[[col]])
    return train_nn, test_nn

In [ ]:
%%time
train_nn, test_nn = apply_quantile(train_pandas, test_pandas)

train_nn['target'] = train_pandas['target']
train_nn['time_id'] = train_pandas['time_id']
train_nn['stock_id'] = train_pandas['stock_id']
test_nn['time_id'] = test_pandas['time_id']
test_nn['stock_id'] = test_pandas['stock_id']

In [ ]:
# kfold based on the knn++ algorithm. It used
class KNN_Fold:
    def __init__(self, n_splits = 5):
        self.nfolds = n_splits
        nfolds = n_splits
        df_train = pandas.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
        out_train = df_train.pivot(index='time_id', columns='stock_id', values='target')

        #out_train[out_train.isna().any(axis=1)]
        out_train = out_train.fillna(out_train.mean())
#         print(out_train.head())

        # code to add the just the read data after first execution

        # data separation based on knn ++
        index = []
        totDist = []
        values = []
        # generates a matriz with the values of 
        mat = out_train.values

        scaler = MinMaxScaler(feature_range=(-1, 1))
        mat = scaler.fit_transform(mat)

        nind = int(mat.shape[0]/nfolds) # number of individuals

        # adds index in the last column
        mat = np.c_[mat,np.arange(mat.shape[0])]


        lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

        lineNumber = np.sort(lineNumber)[::-1]

        for n in range(nfolds):
            totDist.append(np.zeros(mat.shape[0]-nfolds))

        # saves index
        for n in range(nfolds):

            values.append([lineNumber[n]])    


        s=[]
        for n in range(nfolds):
            s.append(mat[lineNumber[n],:])

            mat = np.delete(mat, obj=lineNumber[n], axis=0)

        for n in range(nind-1):    

            luck = np.random.uniform(0,1,nfolds)

            for cycle in range(nfolds):
                 # saves the values of index           

                s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

                sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
                totDist[cycle] += sumDist        

                # probabilities
                f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
                j = 0
                kn = 0
                for val in f:
                    j += val        
                    if (j > luck[cycle]): # the column was selected
                        break
                    kn +=1
                lineNumber[cycle] = kn

                # delete line of the value added    
                for n_iter in range(nfolds):

                    totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
                    j= 0

                s[cycle] = mat[lineNumber[cycle],:]
                values[cycle].append(int(mat[lineNumber[cycle],-1]))
                mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


        for n_mod in range(nfolds):
            values[n_mod] = out_train.index[values[n_mod]]    
        self.values = values
        
    def split(self, train, y_target, time_id):
        nfolds = self.nfolds
        values = self.values


        for fold in range(nfolds):
            indexes = np.arange(nfolds).astype(int)    
            indexes = np.delete(indexes,obj=fold, axis=0) 

            indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
            yield (list(train.time_id.isin(indexes)), list(train.time_id.isin(values[fold])))

## Release GPU for NN

In [ ]:
# del train,test
_= gc.collect()

## Common methods for NN

In [ ]:
#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
    gc.collect()

In [ ]:
from numpy.random import seed
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(42)
from tensorflow import keras
import numpy as nptensorflow
from tensorflow.keras import backend as K
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [ ]:
def root_mean_squared_per_error(y_true, y_pred):
#     tf.print('y_true', y_true)
#     tf.print('y_pred', y_pred)
    return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))


## Autoencoder

In [ ]:
def AE_MLP_model(max_id_num, feature_num, stock_embedding_size, hidden_units, dropout_rates):

    cat_data = train_nn['stock_id']
    print('AE_MLP_model feature number:', feature_num)
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = tf.keras.Input(shape=(1,), name='stock_id')
    num_input = tf.keras.Input(shape=(feature_num,), name='num_data')

    #embedding, flatenning and concatenating
    stock_embedded = tf.keras.layers.Embedding(max_id_num+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = tf.keras.layers.Flatten()(stock_embedded)
    inp = tf.keras.layers.Concatenate()([stock_flattened, num_input])
    x0 = tf.keras.layers.BatchNormalization()(inp)
    x0 = inp
    
    encoder = tf.keras.layers.GaussianNoise(dropout_rates[0])(x0)
    encoder = tf.keras.layers.Dense(hidden_units[0])(encoder)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation('swish')(encoder)
    
    decoder = tf.keras.layers.Dropout(dropout_rates[1])(encoder)
    # decoder is features+stock ids
    decoder = tf.keras.layers.Dense(feature_num+1, name = 'decoder')(decoder)

    x_ae = tf.keras.layers.Dense(hidden_units[1])(decoder)
    x_ae = tf.keras.layers.BatchNormalization()(x_ae)
    x_ae = tf.keras.layers.Activation('swish')(x_ae)
    x_ae = tf.keras.layers.Dropout(dropout_rates[2])(x_ae)
    
    out_ae = tf.keras.layers.Dense(1, activation = 'linear', name = 'ae_prediction')(x_ae)
    
    x = tf.keras.layers.Concatenate()([x0, encoder])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[3])(x)
    
    for i in range(2, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 2])(x)
    
    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(x)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = [decoder
               ,out_ae
               ,out
              ],
    )
    
    model.summary()
    
    return model

In [ ]:
def train_and_evaluate_complex_nn(train, test, params, model_creator):
    
    features = [col for col in train.columns if col not in {"time_id", "target", "target_sqr", "row_id", 'is_train'}]
    y = train['target']
    
    y_train = np.zeros(train.shape[0])
    y_test = np.zeros(test.shape[0])
    
    print('Check null in train', train[features].isnull().sum())
    print('Check null in test', test[features].isnull().sum())
    train[features] = train[features].fillna(train[features].mean())
    test[features] = test[features].fillna(train[features].mean())
    print('Check null in test again', test[features].isnull().sum())
    
#     kf = GroupKFold(n_splits=NFOLD)
    kf = KNN_Fold(n_splits=NFOLD)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, y_target, time_id)):
        print('Fold:', fold)
        x_train, x_val = train.iloc[train_idx], train.iloc[valid_idx]
        y_tra, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        
        cat_data = x_train['stock_id']
        features.remove('stock_id')

        scaler = MinMaxScaler(feature_range=(-1, 1))
        num_data = x_train[features].values
        num_data = scaler.fit_transform(num_data)
        cat_data_val = x_val['stock_id']
        num_data_val = x_val[features].values
        num_data_val = scaler.transform(num_data_val)
        
        # Add stockid to validate decorder
        num_data_with_stockid = np.c_[num_data, cat_data]
        
        # Add stockid to validate decorder
        num_data_test_with_stockid = np.c_[num_data_val, cat_data_val]
        
        model = model_creator(max(cat_data), len(features), params['stock_embedding_size'], 
                             params['hidden_units'], params['dropout_rates'])

        model.compile(
            keras.optimizers.Adam(learning_rate=params['learning_rate']),
            loss={
#                 'decoder': tf.keras.losses.MeanSquaredError()
                'decoder': tf.keras.losses.MeanAbsoluteError()
                  ,'ae_prediction': root_mean_squared_per_error
                  ,'prediction': root_mean_squared_per_error
                 }
        )
        
#         
        es = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=20, verbose=0,
            mode='min',restore_best_weights=True)

        plateau = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2, patience=7, verbose=0,
            mode='min')

        model.fit([cat_data, num_data], 
                  [num_data_with_stockid, y_tra, y_tra],               
                  batch_size=params['batch_size'],
                  epochs=params['epochs'],
                  validation_data=([cat_data_val, num_data_val], [num_data_test_with_stockid, y_val, y_val]),
                  callbacks=[es, 
                            plateau
                            ],
                  validation_batch_size=len(y_val),
                  shuffle=True,
                 verbose = 1)

        pred_val = np.clip(model.predict([cat_data_val, num_data_val])[-1].reshape(1,-1)[0], low_clip, upper_clip)
        y_train[valid_idx] = pred_val
        test_nn = test[features].values
        test_nn = scaler.transform(test_nn)
        y_test += np.clip(model.predict([test['stock_id'], test_nn])[-1].reshape(1,-1)[0], low_clip, upper_clip)
        features.append('stock_id')
        
        print(y_train[valid_idx][:3], y_test[:3])
        print('NN base Rmspe Fold:', rmspe(y_target[valid_idx], y_train[valid_idx]))
        
        #Delete model and release GPU memory
        del model, cat_data, num_data, cat_data_val, num_data_val, scaler, test_nn
        gc.collect()
        reset_keras()
    y_test/=NFOLD
    
    return y_train, y_test

In [ ]:
nn_complex_time = time.time()

params = {
    'batch_size': 4096,
    'epochs': 1000,
    'learning_rate': 1e-3,
    'stock_embedding_size': 8,
    'hidden_units': [96, 96, 128, 64, 32],
    'dropout_rates': [0.03527936123679956,
                     0.038424974585075086,
                     0.42409238408801436,
                     0.10431484318345882,
                     0.49230389137187497,
                     0.32024444956111164,
                     0.2716856145683449,
                     0.4379233941604448]
}

y_ae_train1, y_ae_test1 = train_and_evaluate_complex_nn(train_nn, test_nn, params, AE_MLP_model)
_= gc.collect()

print('Check zero in prediction:', (y_ae_train1 == 0).sum())
np.savetxt('pred_ae.csv', y_ae_train1, delimiter=',')
              
print( 'NN AE Rmspe CV:', rmspe(y_target, y_ae_train1), 'time: ', int(time.time() - nn_complex_time), 's', y_ae_test1[:3])

## Baseline

In [ ]:
def base_model(max_id_num, feature_num, stock_embedding_size, hidden_units, dropout_rates):
    print('base_model', max_id_num, feature_num, stock_embedding_size, hidden_units, dropout_rates)
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(feature_num,), name='num_data')
    input_dense_num = feature_num
    if hidden_units[0] != -1:
        input_dense_num = hidden_units[0]
    
    input_features = keras.layers.Dense(input_dense_num)(num_input)
    
    input_features = keras.layers.BatchNormalization()(input_features)
    input_features = keras.layers.Activation('swish')(input_features)
    if dropout_rates[0] < 1:
        input_features = keras.layers.Dropout(dropout_rates[0])(input_features)

    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max_id_num+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, input_features])
    
#     # Add one or more hidden layers
    for i in range(1, len(hidden_units)):
        out = keras.layers.Dense(hidden_units[i])(out)
        out = keras.layers.BatchNormalization()(out)
        out = keras.layers.Activation('swish')(out)
        if dropout_rates[i] < 1:
            out = keras.layers.Dropout(dropout_rates[i])(out)
        
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out
    )
    
    model.summary()
    
    return model

def train_and_evaluate_nn_base(train, test, params):
    
    features = [col for col in train.columns if col not in {"time_id", "target", "target_sqr", "row_id", 'is_train'}]
    y = train['target']
    
    y_train = np.zeros(train.shape[0])
    y_test = np.zeros(test.shape[0])
    
    print('Check null in train', train[features].isnull().sum())
    print('Check null in test', test[features].isnull().sum())
    train[features] = train[features].fillna(train[features].mean())
    test[features] = test[features].fillna(train[features].mean())
    print('Check null in test again', test[features].isnull().sum())
    
#     kf = GroupKFold(n_splits=NFOLD)
    kf = KNN_Fold(n_splits=NFOLD)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, y_target, time_id)):
        print('Fold:', fold)
        x_train, x_val = train.iloc[train_idx], train.iloc[valid_idx]
        y_tra, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        
        cat_data = x_train['stock_id']
        features.remove('stock_id')

        scaler = MinMaxScaler(feature_range=(-1, 1))
        num_data = x_train[features].values
        num_data = scaler.fit_transform(num_data)
        cat_data_val = x_val['stock_id']
        num_data_val = x_val[features].values
        num_data_val = scaler.transform(num_data_val)
        
        model = base_model(max(cat_data), len(features), params['stock_embedding_size'], params['hidden_units'], params['dropout_rates'])

        model.compile(
            keras.optimizers.Adam(learning_rate=params['learning_rate']),
            loss=root_mean_squared_per_error
        )
        
#         

        es = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=20, verbose=0,
            mode='min',restore_best_weights=True)

        plateau = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2, patience=7, verbose=0,
            mode='min')

        model.fit([cat_data, num_data], 
                  y_tra,               
                  batch_size=params['batch_size'],
                  epochs=params['epochs'],
                  validation_data=([cat_data_val, num_data_val], y_val),
                  callbacks=[es, 
                            plateau
                            ],
                  validation_batch_size=len(y_val),
                  shuffle=True,
                 verbose = 1)

        pred_val = np.clip(model.predict([cat_data_val, num_data_val]).reshape(1,-1)[0], low_clip, upper_clip)
        y_train[valid_idx] = pred_val
        test_nn = test[features].values
        test_nn = scaler.transform(test_nn)
        y_test += np.clip(model.predict([test['stock_id'], test_nn]).reshape(1,-1)[0], low_clip, upper_clip)
        features.append('stock_id')
        
        print(y_train[valid_idx][:3], y_test[:3])
        print('NN base Rmspe Fold:', rmspe(y_target[valid_idx], y_train[valid_idx]))
        
        #Delete model and release GPU memory
        del model, cat_data, num_data, cat_data_val, num_data_val, scaler, test_nn
        gc.collect()
        reset_keras()
    y_test/=NFOLD
    
    
    
    return y_train, y_test



In [ ]:
nn_base_time = time.time()

params = {
    'batch_size': 4096,
    'epochs': 1000,
    'learning_rate': 0.006,
    'stock_embedding_size': 8,
#     'hidden_units': [128, 128, 64, 32], 1.180226882297499
    'hidden_units': [-1, 128, 64, 32],
    'dropout_rates': [0.03, 0.32024444956111164,
                     0.2716856145683449,
                     0.4379233941604448] # 1 means no dropout
}

y_nn_train1, y_nn_test1 = train_and_evaluate_nn_base(train_nn, test_nn, params)
_= gc.collect()

print('Check zero in prediction:', (y_nn_train1 == 0).sum())
np.savetxt('pred_nn.csv', y_nn_train1, delimiter=',')
              
print( 'NN base Rmspe CV:', rmspe(y_target, y_nn_train1), 'time: ', int(time.time() - nn_base_time), 's', y_nn_test1[:3])

# XGBoost GPU

In [ ]:
#%%script echo skipping
xgbtime = time.time()

# Define the custom metric to optimize
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    err = rmspe(labels, preds)
    return 'rmspe', err

def train_and_evaluate_xgb(train, test, params, colNames):
    # Sample weight
    train['target_sqr'] = 1. / (train['target'] ** 1.60 + 9e-7)    
    train.loc[train.stock_id==31,'target_sqr'] = 0.0001

    dtest = xgb.DMatrix(test[colNames])

    y_train = np.zeros(len(train))
    y_test = np.zeros(len(test))

    kf = GroupKFold(n_splits=NFOLD)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, y_target, time_id)):
        print('Fold:', fold)
        dtrain = xgb.DMatrix(train.loc[train_idx, colNames], train.loc[train_idx, 'target']*10000, weight=train.loc[train_idx, 'target_sqr'])
        dvalid = xgb.DMatrix(train.loc[valid_idx, colNames], train.loc[valid_idx, 'target']*10000)
        model = xgb.train(
            params,
            dtrain,
            3000,
            #[(dtrain, "train"), (dvalid, "valid")],
            [(dvalid, "valid")],
            verbose_eval=250,
            early_stopping_rounds=50,
            feval=evalerror,
        )
        y_train[valid_idx] = np.clip(model.predict(dvalid)/10000, low_clip, upper_clip)
        y_test += np.clip((model.predict(dtest))/10000, low_clip, upper_clip)
        print( 'Rmspe Fold:', rmspe(y_target[valid_idx], y_train[valid_idx]) )
    y_test /= NFOLD
    
    print( 'XGBoost Rmspe CV:', rmspe(y_target, y_train) )
    print( pandas.DataFrame.from_dict( model.get_score(), orient='index').sort_values(0, ascending=False).head(20) )
    print()
    
    del model, dtest, dtrain, dvalid
    _ = gc.collect()
    
    return y_train, y_test


colNames = [col for col in list(train.columns) if col not in {'is_train', 'time_id', 'target', 'row_id', 'target_sqr', 'is_train'}]
colNames = [col for col in colNames if col.find('min')<0 ]
params = {
        "subsample": 0.60,
        "colsample_bytree": 0.40,
        "max_depth": 6,
        "learning_rate": 0.02,
        "objective": "reg:squarederror",
        'disable_default_eval_metric': 1, # <- necessary for XGBoost to earlystop by Rmspe and not the default rmse
        "nthread": -1,
        "tree_method": "gpu_hist",
        "gpu_id": 0,
        "max_bin": 128, 
        'min_child_weight': 0,
        'reg_lambda': 0.001,
        'reg_alpha': 0.001, 
        'seed' : 2021,
        'single_precision_histogram': False,
    }
y_train1a, y_test1a = train_and_evaluate_xgb(train, test, params, colNames)


colNames = [col for col in list(train.columns) if col not in {'is_train', 'time_id', 'target', 'row_id', 'target_sqr', 'is_train'}]
colNames = [col for col in colNames if col.find('max')<0 ]
params = {
        "subsample": 0.85,
        "colsample_bytree": 0.25,
        "max_depth": 7,
        "learning_rate": 0.02,
        "objective": "reg:squarederror",
        'disable_default_eval_metric': 1, # <- necessary for XGBoost to earlystop by Rmspe and not the default rmse
        "nthread": -1,
        "tree_method": "gpu_hist",
        "gpu_id": 0,
        "max_bin": 128, 
        'min_child_weight': 0,
        'reg_lambda': 0.001,
        'reg_alpha': 0.001, 
        'seed' : 2022,
        'single_precision_histogram': False,
    }
y_train1b, y_test1b = train_and_evaluate_xgb(train, test, params, colNames)


y_train1 = 0.5*y_train1a + 0.5*y_train1b
y_test1  = 0.5*y_test1a  + 0.5*y_test1b
_= gc.collect()
xgbtime = time.time() - xgbtime

print( 'XGBoost Rmspe CV:', rmspe(y_target, y_train1), 'time: ', int(xgbtime), 's', y_test1[:3] )

# CatBoost

In [ ]:
#%%script echo skipping
catbtime = time.time()

def train_and_evaluate_catb(train, test, params):

    # Sample weight
    train['target_sqr'] = 1. / (train['target'] ** 1.75 + 1e-6)
    train.loc[train.stock_id==31,'target_sqr'] = 1.

    colNames = [col for col in list(train.columns) if col not in {'is_train', 'time_id', 'target', 'row_id', 'target_sqr', 'is_train'}]

    y_train = np.zeros(len(train))
    y_test = np.zeros(len(test))

    kf = GroupKFold(n_splits=NFOLD)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, y_target, time_id)):
        print('Fold:', fold)

        model = CatBoostRegressor(
            iterations=3000,
            learning_rate=0.05,
            depth=7,
            loss_function='RMSE',
            #l2_leaf_reg = 0.001,
            #random_strength = 0.5,
            #bagging_temperature = 1.0,
            task_type="GPU",
            random_seed = 2021,
        )        
        model.fit(
            X=train.loc[train_idx, colNames].to_pandas(), y=train.loc[train_idx, 'target'].to_pandas(),
            sample_weight = train.loc[train_idx, 'target_sqr'].to_pandas(),
            eval_set = (train.loc[valid_idx, colNames].to_pandas(), train.loc[valid_idx, 'target'].to_pandas(),),
            early_stopping_rounds = 20,
            cat_features = [0],
            verbose=False)

        y_train[valid_idx] = np.clip(model.predict(train.loc[valid_idx, colNames].to_pandas()), low_clip, upper_clip)
        y_test += np.clip((model.predict(test[colNames].to_pandas())), low_clip, upper_clip)
        print( 'Catboost Rmspe Fold:', rmspe(y_target[valid_idx], y_train[valid_idx]) )        
        print()
    y_test /= NFOLD
    return y_train, y_test


y_train2, y_test2 = train_and_evaluate_catb(train, test, params)
_= gc.collect()
catbtime = time.time() - catbtime
     
print( 'Catboost Rmspe CV:', rmspe(y_target, y_train2), 'time: ', int(catbtime), 's', y_test2[:3]  )

# LightGBM GPU

In [ ]:
lgbtime = time.time()

# Define the custom metric to optimize
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def train_and_evaluate_lgb(train, test, params):
    
    features = [col for col in train.columns if col not in {"time_id", "target", "target_sqr", "row_id", 'is_train'}]
    y = train['target']
    
    y_train = np.zeros(train.shape[0])
    y_test = np.zeros(test.shape[0])
    
    kf = GroupKFold(n_splits=NFOLD)
    for fold, (trn_ind, val_ind) in enumerate(kf.split(train, y_target, time_id)):
        print('Fold:', fold)
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_tra, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        
        train_dataset = lgb.Dataset(x_train[features], y_tra, weight = (1. / (np.square(y_tra) + 1e-6)) )
        valid_dataset = lgb.Dataset(x_val[features], y_val)
        model = lgb.train(params = params,
                          num_boost_round=3000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, valid_dataset], 
                          verbose_eval = 100,
                          early_stopping_rounds=20,
                          feval = feval_rmspe)
        
        y_train[val_ind] = np.clip(model.predict(x_val[features]), low_clip, upper_clip)
        print('LightGBM Rmspe Fold:', rmspe(y_val, y_train[val_ind]))
        y_test += np.clip((model.predict(test[features])), low_clip, upper_clip)
    y_test/=NFOLD
    
    lgb.plot_importance(model,max_num_features=20)
    
    return y_train, y_test


params = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':255,
    'min_data_in_leaf':750,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 3,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':2021,
    'n_jobs':-1,
    'verbose': -1,
    'device': 'gpu',
    'num_gpu': 1,
    'gpu_platform_id':-1,
    'gpu_device_id':-1,
    'gpu_use_dp': False,
}

y_train3, y_test3 = train_and_evaluate_lgb(train_pandas, test_pandas, params)
_= gc.collect()

print( 'LightGBM Rmspe CV:', rmspe(y_target, y_train3), 'time: ', int(time.time() - lgbtime), 's', y_test3[:3]   )

# Ensembling Time

In [ ]:
%%script echo skipping
print( 'XGBoost Rmspe:', rmspe(y_target, y_train1) )
print( 'CatBoost Rmspe:', rmspe(y_target, y_train2) )
print( 'LightGBM Rmspe:', rmspe(y_target, y_train3) )
print( 'NN base Rmspe:', rmspe(y_target, y_nn_train1) )

In [ ]:
ensemble_trains = [
    y_train1, y_train2, y_train3, 
    y_nn_train1, 
    y_ae_train1
]

ensemble_tests = [
    y_test1, y_test2, y_test3, 
    y_nn_test1, 
    y_ae_test1
]

ENSEMBLING_SIZE = len(ensemble_trains)

In [ ]:
for y_train in ensemble_trains:
    print('Ensemble Rmspe:', rmspe(y_target, y_train) )

In [ ]:
def show_error(train_pandas, ypred0, ypred1):
    train_pandas['ypred'] = (ypred0+ypred1)/2
    train_pandas['error'] = (train_pandas['target'] - train_pandas['ypred']) / train_pandas['target']
    train_pandas['error'] = train_pandas['error']**2

    dt = train_pandas.groupby('stock_id')['error'].agg('mean').reset_index()
    dt['error'] = np.sqrt(dt['error'])
    dt = dt.sort_values('error', ascending=False)
    dt.to_csv('error-contribution.csv', index=False)
    del train_pandas['ypred'], train_pandas['error']
    print(dt.head(10))
    print(dt.tail(10))

    
def calc_arit(W, ensemble_trains):
    #ypred = W[0] * y_train1 + W[1] * y_train2 + W[2] * y_train3 + W[3] * y_nn_train1
    ypred = 0
    for i in range(len(ensemble_trains)):
        ypred = W[i] * ensemble_trains[i] + ypred    
    return ypred

def minimize_arit(W, y_target, ensemble_trains):
    return rmspe(y_target, calc_arit(W, ensemble_trains))

def signed_power(var, p=2):
    return np.sign(var) * np.abs(var)**p

def calc_geom(W, ensemble_trains):
    #ypred = signed_power(y_train1, W[0]) * signed_power(y_train2, W[1]) * signed_power(y_train3, W[2]) * signed_power(y_nn_train1, W[3])
    ypred = 1
    for i in range(len(ensemble_trains)):
        ypred = signed_power(ensemble_trains[i], W[i]) * ypred
    return ypred

def minimize_geom(W, y_target, ensemble_trains):
    return rmspe(y_target, calc_geom(W, ensemble_trains))    

class LinearEnsamble:
    def __init__(self, ensemble_trains, y_target, ensemble_tests):
        self.ensemble_trains = ensemble_trains
        self.y_target = y_target
        self.ensemble_tests = ensemble_tests
        
    
    def fit(self):
        ensemble_trains, y_target, ensemble_tests = self.ensemble_trains, self.y_target, self.ensemble_tests
        
        bounds = [(0, 1) for _ in ensemble_trains]
        
        #avoid negative weights for arit
        W0 = minimize(minimize_arit, [1./ENSEMBLING_SIZE]*ENSEMBLING_SIZE, options={'gtol': 1e-6, 'disp': True},
                 args=(y_target, ensemble_trains), bounds=bounds).x
        print('Weights arit:',W0)
        
        #Geom weight can be negative
        W1 = minimize(minimize_geom, [1./ENSEMBLING_SIZE]*ENSEMBLING_SIZE, options={'gtol': 1e-6, 'disp': True},
                     args=(y_target, ensemble_trains)).x

        print('weights geom:',W1)
        
        ypred0 = calc_arit(W0, ensemble_trains)
        print( np.min(ypred0), np.mean(ypred0), np.max(ypred0), np.std(ypred0))

        ypred1 = calc_geom(W1, ensemble_trains)
        print( np.min(ypred1), np.mean(ypred1), np.max(ypred1), np.std(ypred1))

        print( 'Ensemble:', rmspe(y_target, (ypred0+ypred1)/2))
        
        print( np.min(ypred0),np.mean(ypred0),np.max(ypred0),np.std(ypred0) )
        print( np.min(ypred1),np.mean(ypred1),np.max(ypred1),np.std(ypred1) )
        
        plt.hist(ypred0, bins=100)
        plt.hist(ypred1, bins=100, alpha=0.5)
        
        self.W0 = W0
        self.W1 = W1
        
        return ypred0, ypred1
        
    def predict(self):
        print(self.ensemble_tests)
        ypred0 = calc_arit(self.W0, self.ensemble_tests)
        ypred1 = calc_geom(self.W1, self.ensemble_tests)

        ypredtest = (ypred0+ypred1)/2
        print( ypred0[:3],  ypred1[:3], ypredtest[:3] )
        
        print(ypredtest)
        return ypredtest

In [ ]:
linearEnsamble = LinearEnsamble(ensemble_trains, y_target, ensemble_tests)
ypred0, ypred1 = linearEnsamble.fit()

show_error(train_pandas, ypred0, ypred1)

ypredtest = linearEnsamble.predict()
print( ypred0[:3],  ypred1[:3], ypredtest[:3] )

test_pandas['target'] = ypredtest
test_pandas[['row_id', 'target']].to_csv('submission.csv',index = False)
test_pandas[['row_id', 'target']].head(3)